# Лабораторная работа №2. Логистическая и Линейная регрессия

**Цель:** Исследование линейных моделей на задачах классификации и регрессии, реализация алгоритмов с нуля.


## 1. Выбор данных

Используются те же датасеты, что и в ЛР1:
1. **Mushrooms** (Классификация) -> Логистическая регрессия.
2. **Car Price** (Регрессия) -> Линейная регрессия.


In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    mean_absolute_error,
    mean_squared_error,
    r2_score
)

mush = pd.read_csv("data/mushrooms.csv")
cars = pd.read_csv("data/car_price.csv")

print("Mushrooms shape:", mush.shape)
print("Cars shape:", cars.shape)

display(mush.head())
display(cars.head())


Mushrooms shape: (8124, 23)
Cars shape: (1000, 8)


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


,Make,Model,Year,Engine Size,Mileage,Fuel Type,Transmission,Price
0,Honda,Model B,2015,3.9,74176,Petrol,Manual,30246.207931
1,Ford,Model C,2014,1.7,94799,Electric,Automatic,22785.747684
2,BMW,Model B,2006,4.1,98385,Electric,Manual,25760.290347
3,Honda,Model B,2015,2.6,88919,Electric,Automatic,25638.003491
4,Honda,Model C,2004,3.4,138482,Petrol,Automatic,21021.386657


In [2]:
X_mush = mush.drop("class", axis=1)
y_mush = mush["class"]

cat_features_mush = X_mush.columns.tolist()

X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_mush,
    y_mush,
    test_size=0.2,
    random_state=42,
    stratify=y_mush
)

print("Mushrooms train/test:", X_train_m.shape, X_test_m.shape)


TARGET_COL = "Price"

X_cars = cars.drop(TARGET_COL, axis=1)
y_cars = cars[TARGET_COL]

num_features_cars = X_cars.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_features_cars = X_cars.select_dtypes(include=["object"]).columns.tolist()

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_cars,
    y_cars,
    test_size=0.2,
    random_state=42
)

print("Cars train/test:", X_train_c.shape, X_test_c.shape)
print("Numeric features:", num_features_cars)
print("Categorical features:", cat_features_cars)


Mushrooms train/test: (6499, 22) (1625, 22)
Cars train/test: (800, 7) (200, 7)
Numeric features: ['Year', 'Engine Size', 'Mileage']
Categorical features: ['Make', 'Model', 'Fuel Type', 'Transmission']


In [3]:
# Препроцессор для грибов: OneHotEncode всех категориальных признаков
preprocess_mush = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features_mush)
    ]
)

# Препроцессор для автомобилей: числовые + категориальные
preprocess_cars = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_features_cars),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features_cars)
    ]
)


In [4]:
log_reg_baseline = Pipeline(steps=[
    ("preprocess", preprocess_mush),
    ("model", LogisticRegression(max_iter=1000))
])

log_reg_baseline.fit(X_train_m, y_train_m)

y_pred_m_log = log_reg_baseline.predict(X_test_m)

print("=== Logistic Regression Baseline (Mushrooms) ===")
print("Accuracy:", accuracy_score(y_test_m, y_pred_m_log))
print("\nClassification report:")
print(classification_report(y_test_m, y_pred_m_log))
print("\nConfusion matrix:")
print(confusion_matrix(y_test_m, y_pred_m_log))


=== Logistic Regression Baseline (Mushrooms) ===
Accuracy: 0.9993846153846154

Classification report:
              precision    recall  f1-score   support

           e       1.00      1.00      1.00       842
           p       1.00      1.00      1.00       783

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625


Confusion matrix:
[[842   0]
 [  1 782]]


In [5]:
lin_reg_baseline = Pipeline(steps=[
    ("preprocess", preprocess_cars),
    ("model", LinearRegression())
])

lin_reg_baseline.fit(X_train_c, y_train_c)

y_pred_c_lin = lin_reg_baseline.predict(X_test_c)

mae_lin = mean_absolute_error(y_test_c, y_pred_c_lin)
rmse_lin = mean_squared_error(y_test_c, y_pred_c_lin) ** 0.5
r2_lin = r2_score(y_test_c, y_pred_c_lin)

print("=== Linear Regression Baseline (Cars) ===")
print("MAE:", mae_lin)
print("RMSE:", rmse_lin)
print("R^2:", r2_lin)


=== Linear Regression Baseline (Cars) ===
MAE: 1810.5547575776332
RMSE: 2237.2910425919263
R^2: 0.8170961815663225


## 4. Анализ результатов бейзлайна логистической и линейной регрессии

### 4.1. Классификация (Mushrooms) — Logistic Regression

Бейзлайн логистической регрессии показал следующие результаты:

- **Accuracy ≈ 0.99938**
- Ошибка всего в одном объекте (`1` неверная классификация)
- Precision/Recall/F1-score для обоих классов практически равны 1.00
- Матрица ошибок показывает полное отсутствие ошибок для съедобного класса и одну ошибку для ядовитого

Интерпретация:

- Логистическая регрессия, несмотря на линейную природу, отлично работает на этом датасете.
- Единственная ошибка связана с тем, что модель пытается провести линейную границу в пространстве признаков, которое сильно расширено после One-Hot кодирования.
- Тем не менее результат практически идеальный и почти не уступает KNN.

---

### 4.2. Регрессия (Car Price Prediction) — Linear Regression

Результаты линейной регрессии:

- **MAE ≈ 1810.55**
- **RMSE ≈ 2237.29**
- **R² ≈ 0.8171**

Интерпретация:

- Линейная модель объясняет более **81%** вариации цен автомобилей.
- Качество **заметно лучше**, чем у KNN-бейзлайна (у KNN было R² ≈ 0.7156).
- MAE около 1800 означает, что модель ошибается в среднем на 1.8 тыс. единиц цены, что является неплохим результатом для простой линейной зависимости.
- Улучшения ожидаются при добавлении регуляризации (`Ridge`) и подборе гиперпараметров.

---

### Итог по бейзлайну

- Логистическая регрессия на грибах показывает почти идеальное качество, что подтверждает хорошую линейную разделимость после One-Hot кодирования.
- Линейная регрессия демонстрирует высокое качество на задаче предсказания цены автомобиля, превосходя результаты KNN.
- Оба бейзлайна дают сильную стартовую точку для дальнейших улучшений.


In [6]:
# Улучшенная логистическая регрессия для грибов (подбор C)

log_reg_pipe = Pipeline(steps=[
    ("preprocess", preprocess_mush),
    ("model", LogisticRegression(max_iter=1000, solver="liblinear"))
])

param_grid_log = {
    "model__C": [0.01, 0.1, 1.0, 10.0, 100.0],
    "model__penalty": ["l1", "l2"]
}

grid_log = GridSearchCV(
    estimator=log_reg_pipe,
    param_grid=param_grid_log,
    cv=5,
    scoring="f1_macro",
    n_jobs=-1
)

grid_log.fit(X_train_m, y_train_m)

print("Best params (LogReg Mushrooms):", grid_log.best_params_)

best_log_clf = grid_log.best_estimator_

y_pred_m_log_best = best_log_clf.predict(X_test_m)

print("\n=== Improved Logistic Regression (Mushrooms) ===")
print("Accuracy:", accuracy_score(y_test_m, y_pred_m_log_best))
print("\nClassification report:")
print(classification_report(y_test_m, y_pred_m_log_best))
print("\nConfusion matrix:")
print(confusion_matrix(y_test_m, y_pred_m_log_best))


Best params (LogReg Mushrooms): {'model__C': 10.0, 'model__penalty': 'l1'}

=== Improved Logistic Regression (Mushrooms) ===
Accuracy: 1.0

Classification report:
              precision    recall  f1-score   support

           e       1.00      1.00      1.00       842
           p       1.00      1.00      1.00       783

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625


Confusion matrix:
[[842   0]
 [  0 783]]


In [7]:
from sklearn.linear_model import Ridge

ridge_pipe = Pipeline(steps=[
    ("preprocess", preprocess_cars),
    ("model", Ridge())
])

param_grid_ridge = {
    "model__alpha": [0.01, 0.1, 1.0, 10.0, 100.0]
}

grid_ridge = GridSearchCV(
    estimator=ridge_pipe,
    param_grid=param_grid_ridge,
    cv=5,
    scoring="neg_mean_absolute_error",
    n_jobs=-1
)

grid_ridge.fit(X_train_c, y_train_c)

print("Best params (Ridge Cars):", grid_ridge.best_params_)

best_ridge_reg = grid_ridge.best_estimator_

y_pred_c_ridge = best_ridge_reg.predict(X_test_c)

mae_ridge = mean_absolute_error(y_test_c, y_pred_c_ridge)
rmse_ridge = mean_squared_error(y_test_c, y_pred_c_ridge) ** 0.5
r2_ridge = r2_score(y_test_c, y_pred_c_ridge)

print("\n=== Improved Ridge Regression (Cars) ===")
print("MAE:", mae_ridge)
print("RMSE:", rmse_ridge)
print("R^2:", r2_ridge)


Best params (Ridge Cars): {'model__alpha': 10.0}

=== Improved Ridge Regression (Cars) ===
MAE: 1807.8404600381991
RMSE: 2232.907746289107
R^2: 0.8178121691156635


## 5. Анализ улучшенного бейзлайна логистической и линейной регрессии

### 5.1. Классификация (Mushrooms) — Logistic Regression (подбор C и penalty)

GridSearchCV нашёл оптимальные параметры:

- **C = 10**
- **penalty = l1**
- solver = liblinear

Результаты улучшенной модели:

- **Accuracy = 1.0**
- F1-score = 1.00 для обоих классов
- Матрица ошибок без единой ошибки

Как и в первой лабораторной работе, датасет грибов показывает практически идеальную линейную разделимость после One-Hot кодирования.  
Гиперпараметры влияют на регуляризацию, но **качество не меняется**, поскольку базовая модель уже близка к максимуму.

**Вывод:** улучшенный бейзлайн подтверждает, что логистическая регрессия полностью решает задачу классификации для данного набора данных.

---

### 5.2. Регрессия (Car Price Prediction) — Ridge Regression (подбор alpha)

Лучшие параметры по результатам GridSearchCV:

- **alpha = 10**

Результаты модели:

- **MAE ≈ 1807.84**  
- **RMSE ≈ 2232.91**  
- **R² ≈ 0.81781**

Сравнение с бейзлайном линейной регрессии:

| Модель                       | MAE      | RMSE     | R²       |
|-----------------------------|----------|----------|----------|
| Linear Regression (baseline) | 1810.55  | 2237.29  | 0.81710  |
| Ridge Regression (improved)  | **1807.84** | **2232.91** | **0.81781** |

Интерпретация:

- Регуляризация L2 (Ridge) слегка снижает ошибку и немного повышает R².
- Это ожидаемый результат: Ridge уменьшает переобучение на признаках, которые после One-Hot кодирования становятся многомерными.
- Улучшение небольшое, но стабильное.

**Вывод:** улучшенный бейзлайн демонстрирует, что регуляризация помогает модели немного лучше обобщать данные, что особенно важно при большом количестве категориальных признаков.

---

### 5.3. Общие выводы

- Логистическая регрессия на грибах достигает идеального качества; подбор гиперпараметров подтверждает устойчивость модели.
- Для задачи прогнозирования цен автомобилей регуляризация Ridge даёт небольшое, но закономерное улучшение качества.
- В отличие от KNN в первой лабораторной работе, линейные модели оказываются сильными бейзлайнами, особенно для регрессии.


In [8]:
# Кодирование признаков для грибов
X_train_m_enc = preprocess_mush.fit_transform(X_train_m)
X_test_m_enc = preprocess_mush.transform(X_test_m)

# Кодирование признаков для автомобилей
X_train_c_enc = preprocess_cars.fit_transform(X_train_c)
X_test_c_enc = preprocess_cars.transform(X_test_c)

print("Encoded mushrooms:", X_train_m_enc.shape, X_test_m_enc.shape)
print("Encoded cars:", X_train_c_enc.shape, X_test_c_enc.shape)


Encoded mushrooms: (6499, 117) (1625, 117)
Encoded cars: (800, 18) (200, 18)


In [11]:
import numpy as np

class MyLinearRegression:
    def __init__(self):
        self.w = None
        self.bias = None 
    
    def _to_dense(self, X):
        if hasattr(X, "toarray"):
            X = X.toarray()
        return np.array(X)
    
    def fit(self, X, y):
        """
        Обучение линейной регрессии по нормальному уравнению.
        Добавляем столбец единиц для bias.
        """
        X = self._to_dense(X)
        y = np.array(y).reshape(-1, 1)
        
        X_bias = np.hstack([np.ones((X.shape[0], 1)), X])
        
        # (X^T X)^{-1} X^T y
        XtX = X_bias.T @ X_bias
        Xty = X_bias.T @ y
        
        lambda_identity = 1e-8 * np.eye(XtX.shape[0])
        self.w = np.linalg.inv(XtX + lambda_identity) @ Xty 
        return self
    
    def predict(self, X):
        """
        Предсказание: y = Xw
        """
        X = self._to_dense(X)
        X_bias = np.hstack([np.ones((X.shape[0], 1)), X])
        y_pred = X_bias @ self.w
        return y_pred.ravel()


In [12]:
my_lin_reg = MyLinearRegression()
my_lin_reg.fit(X_train_c_enc, y_train_c)

y_pred_c_my_lin = my_lin_reg.predict(X_test_c_enc)

mae_my_lin = mean_absolute_error(y_test_c, y_pred_c_my_lin)
rmse_my_lin = mean_squared_error(y_test_c, y_pred_c_my_lin) ** 0.5
r2_my_lin = r2_score(y_test_c, y_pred_c_my_lin)

print("=== MyLinearRegression (Cars) ===")
print("MAE:", mae_my_lin)
print("RMSE:", rmse_my_lin)
print("R^2:", r2_my_lin)

=== MyLinearRegression (Cars) ===
MAE: 1810.5583604371268
RMSE: 2237.2762333632504
R^2: 0.8170986029370397


In [14]:
class MyLogisticRegression:
    def __init__(self, lr=0.1, n_iter=1000):
        self.lr = lr
        self.n_iter = n_iter
        self.w = None
    
    def _to_dense(self, X):
        if hasattr(X, "toarray"):
            X = X.toarray()
        return np.array(X)
    
    def _sigmoid(self, z):
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))
    
    def fit(self, X, y):
        """
        Обучение логистической регрессии с помощью градиентного спуска.
        """
        X = self._to_dense(X)
        y = np.array(y).reshape(-1, 1)
        
        X_bias = np.hstack([np.ones((X.shape[0], 1)), X])
        n_samples, n_features = X_bias.shape
        
        self.w = np.zeros((n_features, 1))
        
        for _ in range(self.n_iter):
            logits = X_bias @ self.w
            y_prob = self._sigmoid(logits)
            grad = (1 / n_samples) * (X_bias.T @ (y_prob - y))
            self.w -= self.lr * grad
        
        return self
    
    def predict_proba(self, X):
        """
        Возвращает вероятности класса 1.
        """
        X = self._to_dense(X)
        X_bias = np.hstack([np.ones((X.shape[0], 1)), X])
        logits = X_bias @ self.w
        return self._sigmoid(logits).ravel()
    
    def predict(self, X, threshold=0.5):
        """
        Предсказание меток класса: 0 или 1.
        """
        probs = self.predict_proba(X)
        return (probs >= threshold).astype(int)


In [15]:
y_train_m_bin = (y_train_m == "p").astype(int)
y_test_m_bin = (y_test_m == "p").astype(int)

print("Class distribution in train (binary):")
print(np.bincount(y_train_m_bin))


Class distribution in train (binary):
[3366 3133]


In [ ]:
my_log_reg = MyLogisticRegression(lr=0.1, n_iter=2000)

my_log_reg.fit(X_train_m_enc, y_train_m_bin)

y_pred_m_my_bin = my_log_reg.predict(X_test_m_enc)

y_pred_m_my = np.where(y_pred_m_my_bin == 1, "p", "e")

print("=== MyLogisticRegression (Mushrooms) ===")
print("Accuracy:", accuracy_score(y_test_m, y_pred_m_my))
print("\nClassification report:")
print(classification_report(y_test_m, y_pred_m_my))
print("\nConfusion matrix:")
print(confusion_matrix(y_test_m, y_pred_m_my))


=== MyLogisticRegression (Mushrooms) ===
Accuracy: 0.9956923076923077

Classification report:
              precision    recall  f1-score   support

           e       0.99      1.00      1.00       842
           p       1.00      0.99      1.00       783

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625


Confusion matrix:
[[842   0]
 [  7 776]]


## 6. Анализ собственной реализации моделей

### 6.1. Линейная регрессия (MyLinearRegression)

Результаты собственной реализации:

- **MAE ≈ 1810.56**
- **RMSE ≈ 2237.28**
- **R² ≈ 0.81710**

Сравнение с базовой моделью `LinearRegression` из sklearn:

| Модель                     | MAE      | RMSE     | R²       |
|----------------------------|----------|----------|----------|
| sklearn LinearRegression   | 1810.55  | 2237.29  | 0.81710  |
| MyLinearRegression         | 1810.56  | 2237.28  | 0.81710  |

Совпадение метрик подтверждает корректность реализации.  
Использование нормального уравнения позволяет полностью повторить поведение базовой линейной регрессии без регуляризации.

---

### 6.2. Логистическая регрессия (MyLogisticRegression)

Результаты собственной реализации:

- **Accuracy ≈ 0.99569**
- Класс `e` классифицирован без ошибок.
- Класс `p` имеет 7 ошибок вместо 1 в baseline sklearn-модели.
- Матрица ошибок:
[[842 0]
[ 7 776]]


Сравнение со sklearn LogisticRegression:

| Модель                          | Accuracy |
|----------------------------------|----------|
| sklearn LogisticRegression       | 0.99938  |
| MyLogisticRegression             | 0.99569  |

Собственная реализация уступает sklearn из-за:
- отсутствия регуляризации,
- простого градиентного спуска,
- фиксированного шага обучения,
- меньшей точности оптимизации.

Тем не менее модель обучается корректно и демонстрирует ожидаемое поведение логистической регрессии с вероятностным разделением классов.

---

### 6.3. Итог по собственным моделям

- Линейная регрессия полностью совпадает с результатами sklearn, подтверждая корректность алгоритма.
- Логистическая регрессия показывает высокое качество, но проигрывает оптимизированной sklearn-реализации.
- В обоих случаях собственные модели являются валидными и работают в соответствии с теоретическими ожиданиями.



## 7. Итоговое сравнение моделей и выводы по лабораторной работе №2

### 7.1. Классификация (Mushrooms)

| Модель                                 | Accuracy |
|----------------------------------------|----------|
| Logistic Regression (baseline)         | 0.99938  |
| Logistic Regression (improved, C=10)   | 1.00000  |
| MyLogisticRegression                   | 0.99569  |

Выводы:

- Логистическая регрессия обучается практически идеально и достигает полной точности при подборе гиперпараметров.
- Собственная реализация логистической регрессии уступает, но также показывает высокое качество.
- Датасет остаётся полностью разделимым после One-Hot кодирования, что объясняет стабильность результатов.

---

### 7.2. Регрессия (Car Price Prediction)

| Модель                       | MAE      | RMSE     | R²       |
|-----------------------------|----------|----------|----------|
| Linear Regression (baseline) | 1810.55  | 2237.29  | 0.81710  |
| Ridge Regression (improved)   | 1807.84  | 2232.91  | 0.81781  |
| MyLinearRegression            | 1810.56  | 2237.28  | 0.81710  |

Выводы:

- Линейная регрессия даёт высокое качество и превосходит KNN из ЛР1.
- Регуляризация Ridge обеспечивает небольшое, но устойчивое улучшение метрик.
- Собственная реализация линейной регрессии полностью совпадает со sklearn.

---

### 7.3. Общие выводы

1. Линейные модели показали высокую эффективность как в классификации, так и в регрессии.
2. Улучшение гиперпараметров влияет на качество, но в пределах ожидаемого: логистическая регрессия становится идеально точной, Ridge немного улучшает ошибки.
3. Реализованные вручную модели подтверждают корректность математических основ алгоритмов.
4. Лабораторная работа выполнена: создана база, улучшенные версии и собственные имплементации, проведено корректное сравнение.

